# 中文分词工具使用实践
中文分词原理的简单总结可以参见[文章](https://cuiqingcai.com/5844.html)。常用的中文分词工具有：结巴分词，StandfordNLP，SnowNLP，哈工大语言云（LTP），THULAC，NLPIR。他们的信息总结如下：


In [1]:
sentences = ['我来到北京清华大学', 
             '工信处女干事每月经过下属科室都要亲口交代24口交换机等技术性器件的安装工作',
            '探访“金特会”举办当天越南河内街头景象',
            '西青区一“路怒女”早高峰别停公交车致多名乘客受伤！']

cantonese_sents = ['西環出盡洪荒之力，林鄭月娥最終提名票是579，難免令外界跌眼鏡。',
                   '唔知係咪林鄭哋公關聽到',
                   '垃圾小圈子，當睇馬騮戲 咁班犯民狗又會玩馬騮戲嘅？',
                   '大6戲一場。',
                   '如果林鄭冇反口覆舌, 攞572票嘅會係葉劉']

## jieba
jieba分词有三种模式：精确模式，全模式和搜索模式。精确模式尝试尽可能正确的将句子分开。全模式则是将所有可能的词语都打印出来；搜索模式则在精确模式的基础上，尝试继续分割长词，因此适合搜索。jieba还基于python的multiprocessing模块实现了并行分词，从而大幅度提升了分词的效率。另外，对于繁体分词，提供了一份支持繁体分词更好的词典文件。并且，jieba还支持命令行文件分词。

从测试结果可以观察到：
+ 对于简体中文，jieba分词的分词结果还是很不错的。特别是对于第二个拥有非常多歧义词语的句子，也可以正确分词。
+ 对于粤语来说，如果比较偏向简体中文分词结果比较好。但是粤语方言严重的句子，分词的结果就非常差，偏向于将词语分为单个字。
+ 并且粤语很多替换用词比如”大6“指”大陆“，这种也非常的难分词
+ 精确模式下默认使用HMM，也没有很好的识别未登录词。非HMM模式，由于某些粤语方言在训练集中很少出现，所以每个词都变成了单字

In [4]:
import jieba

for sent in sentences:
    print("精确模式: %s" % " ".join(jieba.cut(sent)))
#     print("全模式: %s" % " ".join(jieba.cut(sent, cut_all=True)))
#     print("搜索模式: %s" % " ".join(jieba.cut_for_search(sent)))
    print("----------------------")

print("========= 粤语")
for sent in cantonese_sents:
    print("精确模式: %s" % " ".join(jieba.cut(sent)))
#     print("非HMM模式: %s" % " ".join(jieba.cut(sent, HMM=False))) 
#     print("全模式: %s" % " ".join(jieba.cut(sent, cut_all=True)))
#     print("搜索模式: %s" % " ".join(jieba.cut_for_search(sent)))
    print("----------------------")


精确模式: 我 来到 北京 清华大学
----------------------
精确模式: 工信处 女干事 每月 经过 下属 科室 都 要 亲口 交代 24 口 交换机 等 技术性 器件 的 安装 工作
----------------------
精确模式: 探访 “ 金特会 ” 举办 当天 越南 河内 街头 景象
----------------------
精确模式: 西青区 一 “ 路怒 女 ” 早 高峰 别停 公交车 致 多名 乘客 受伤 ！
----------------------
========= 粤语
精确模式: 西環出 盡 洪荒 之力 ， 林 鄭月 娥 最終 提名 票 是 579 ， 難免 令 外界 跌 眼鏡 。
----------------------
精确模式: 唔 知 係 咪 林 鄭 哋 公關 聽 到
----------------------
精确模式: 垃圾 小圈子 ， 當睇 馬 騮 戲   咁 班 犯民狗 又 會 玩馬騮 戲 嘅 ？
----------------------
精确模式: 大 6 戲一場 。
----------------------
精确模式: 如果 林鄭 冇 反口 覆舌 ,   攞 572 票 嘅 會 係 葉 劉
----------------------


+ 对于一些有歧义的词语，自定义词典可以提高分词效果（如最后一个句子中的"路怒女"）
+ 增加词典对于粤语分词的提升效果也是很明显的，可以看到增加到词典中的词都被正确划分了
+ 但是这种方法一方面需要增加大量的词来提高分词正确率，另一方面分词的结果极大的依赖于自定义的词典

In [3]:
jieba.add_word("路怒女")
print(" ".join(jieba.cut(sentences[-1])))

西青区 一 “ 路怒女 ” 早 高峰 别停 公交车 致 多名 乘客 受伤 ！


In [4]:
add_dict = ["西環", "洪荒之力", "林鄭月娥", "馬騮戲", "大6", "林鄭", "反口覆舌", "葉劉"]
for w in add_dict:
    jieba.add_word(w)
    
for sent in cantonese_sents:
    print("精确模式: %s" % " ".join(jieba.cut(sent)))
    print("----------------------")


精确模式: 西環 出盡 洪荒之力 ， 林鄭月娥 最終 提名 票 是 579 ， 難免 令 外界 跌 眼鏡 。
----------------------
精确模式: 唔 知 係 咪 林鄭 哋 公關 聽 到
----------------------
精确模式: 垃圾 小圈子 ， 當睇 馬騮戲   咁 班 犯民狗 又 會 玩 馬騮戲 嘅 ？
----------------------
精确模式: 大6 戲一場 。
----------------------
精确模式: 如果 林鄭 冇 反口覆舌 ,   攞 572 票 嘅 會 係 葉劉
----------------------


## snowNLP
[SnowNLP](https://github.com/isnowfy/snownlp)正确来说是一个类库，其中包括了分词，关键词提取，简繁体转换，拼音标注，文本分类等多种功能。其中，SnowNLP的分词采用[基于字的生成模型](http://aclweb.org/anthology//Y/Y09/Y09-2047.pdf)，期望在集内和集外词语上都达到较好的分词效果。

从如下分词结果中可以观察到，SnowNLP的分词效果总体一般：
+ 对于简体中文来说，常见的一些词语也分错了，比如”'工', '信处女', '干事'“， ”'公交', '车致'“等。并且不常见的”金特会“词语也没有分对。
+ 对于繁体来说，其使用的基于字的生成模型在集外词上的表现很差，可以说将词语都分为了单子。并且”洪荒之力“，”跌眼镜“等词也没有分对，一方面说明其训练集包含的词语较少，另一方面从其上一次更新是2年前也可以窥得一斑。
+ 并且SnowNLP好像还不提供自定义词典的功能
+ 另外，虽然SnowNLP提供了多种功能，但是从其issues中可以看出分词之外的其他功能的效果也不是很好，例如关键词提取功能的示例中最关键的词语应该是”自然语言处理“而不是”自然“，”语言“； 对于情感分析，不支持否定词；拼音标注并不提供音调。

In [10]:
from snownlp import SnowNLP

for sent in sentences:
    ret = SnowNLP(sent)
    print(ret.words)

print("----------")
for sent in cantonese_sents:
    ret = SnowNLP(sent)
    print(ret.words)  

['我', '来到', '北京', '清华大学']
['工', '信处女', '干事', '每月', '经过', '下属', '科室', '都', '要', '亲口', '交代', '24', '口', '交换机', '等', '技术性', '器件', '的', '安装', '工作']
['探访', '“', '金特', '会', '”', '举办', '当天', '越南', '河内', '街头', '景象']
['西青区', '一', '“', '路', '怒', '女', '”', '早', '高峰', '别', '停', '公交', '车致', '多', '名', '乘客', '受伤', '！']
----------
['西', '環出', '盡', '洪', '荒', '之', '力', '，', '林', '鄭', '月', '娥', '最', '終', '提', '名', '票', '是', '579，', '難免', '令', '外界', '跌', '眼', '鏡', '。']
['唔', '知', '係', '咪', '林', '鄭', '哋', '公', '關', '聽', '到']
['垃圾', '小圈子', '，', '當', '睇', '馬', '騮', '戲', '咁班', '犯民', '狗', '又', '會', '玩', '馬', '騮', '戲', '嘅', '？']
['大', '6', '戲', '一', '場', '。']
['如', '果', '林', '鄭', '冇', '反口', '覆舌', ',', '攞', '572', '票', '嘅', '會', '係', '葉', '劉']


## pynlpir
[pynlpir](https://github.com/tsroten/pynlpir)是[NLPIR-ICTCLAS](http://ictclas.nlpir.org/)的python版本。NLPIR-ICTCLAS由张华平博士领导开发。pynlpir支持用户自定义词典，以及文件分词。

ictclas采用的分词原理是N最短路径方法，并且再此基础上通过HMM识别未登录词，来优化分词结果，最后根据再次根据HMM的词性标注结果区分人名和叠词，进一步优化。开源的ictclas代码:https://github.com/muyangquan/ictclas

pynlpir已经有2年没有更新，而nlpir工具虽然非商用版本可以永久免费，但是需要经常更新license，使用很不方便。

从下面的测试结果来看，pynlpir的分词结果不是很好，而且划分结果偏向单字。
+ 对于简体中文来说，出现了多个错误划分。比如“'工', '信', '处女', '干事'”, "'公交', '车'"等。而且总体的分词结果来看，pynlpir倾向于将词语尽可能的细粒度划分。
+ 对于繁体，总体可以说是没有分词，几乎每个词语都划分为了单字。但是却正确划分了”洪荒之力“， ”路怒女“， ”馬騮戲“等较难区分的词。
+ 而且最不可思议的是最后一句话的最后一个字被删除掉了。

In [9]:
import pynlpir
pynlpir.open()

for sent in sentences:
    print(pynlpir.segment(sent, pos_tagging=False))

for sent in cantonese_sents:
    print(pynlpir.segment(sent, pos_tagging=False))
pynlpir.close()

['我', '来到', '北京', '清华大学']
['工', '信', '处女', '干事', '每月', '经过', '下属', '科室', '都', '要', '亲口', '交代', '24', '口', '交换机', '等', '技术性', '器件', '的', '安装', '工作']
['探访', '“', '金特', '会', '”', '举办', '当天', '越南', '河内', '街头', '景象']
['西青区', '一', '“', '路怒女', '”', '早', '高峰', '别', '停', '公交', '车', '致', '多', '名', '乘客', '受伤', '！']
['西', '環', '出', '盡', '洪荒之力', '，', '林鄭月娥', '最', '終', '提名', '票', '是', '579', '，', '難', '免', '令', '外界', '跌', '眼', '鏡', '。']
['唔', '知', '係咪', '林鄭', '哋', '公', '關', '聽', '到']
['垃圾', '小圈子', '，', '當', '睇', '馬騮戲', ' ', '咁', '班', '犯', '民', '狗', '又', '會', '玩', '馬騮戲', '嘅', '？']
['大', '6', '戲', '一', '場', '。']
['如果', '林鄭', '冇', '反', '口', '覆', '舌', ',', ' ', '攞', '572', '票', '嘅', '會', '係', '葉']


可以观察到:加在了用户词典后分词结果并没有完全按照用户自定义词典划分，例如”'工', '信', '处女'“依然划分错误。根据[网上的资料](https://www.zhihu.com/question/265456264)显示，pynlpir并不强制让分词结果按照自定义词典划分，而是将它们作为未登录词参与其他普通词语的概率竞争。


In [10]:
# 公交车
# 路怒女
# 洪荒之力
# 林鄭月娥
# 林鄭
# 馬騮戲
# 葉劉
# 係咪
# 工信处
# 金特会
# 跌眼鏡

pynlpir.open()
pynlpir.nlpir.ImportUserDict(b'user_dict.txt')
for sent in sentences:
    print(pynlpir.segment(sent, pos_tagging=False))

for sent in cantonese_sents:
    print(pynlpir.segment(sent, pos_tagging=False))
pynlpir.close()

['我', '来到', '北京', '清华大学']
['工', '信', '处女', '干事', '每月', '经过', '下属', '科室', '都', '要', '亲口', '交代', '24', '口', '交换机', '等', '技术性', '器件', '的', '安装', '工作']
['探访', '“', '金特会', '”', '举办', '当天', '越南', '河内', '街头', '景象']
['西青区', '一', '“', '路怒女', '”', '早', '高峰', '别', '停', '公交', '车', '致', '多', '名', '乘客', '受伤', '！']
['西', '環', '出', '盡', '洪荒之力', '，', '林鄭月娥', '最', '終', '提名', '票', '是', '579', '，', '難', '免', '令', '外界', '跌眼鏡', '。']
['唔', '知', '係咪', '林鄭', '哋', '公', '關', '聽', '到']
['垃圾', '小圈子', '，', '當', '睇', '馬騮戲', ' ', '咁', '班', '犯', '民', '狗', '又', '會', '玩', '馬騮戲', '嘅', '？']
['大', '6', '戲', '一', '場', '。']
['如果', '林鄭', '冇', '反', '口', '覆', '舌', ',', ' ', '攞', '572', '票', '嘅', '會', '係', '葉']


## thulac
[THULAC](https://github.com/thunlp/THULAC)是由清华大学自然语言处理与社会人文计算实验室推出的一套中文词法分析工具包，提供分词和词性标注功能。thulac包含c++版本、java版本和python版本，以及动态库版本（对应fast接口）。另外，该工具还支持模型训练，可以自己下载模型并且在自己的训练数据集上训练。`cut_f()`支持知识对文件进行分词。并且可以直接运行python命令在命令行完成分词。

测试使用的thulac版本：

In [7]:
import thulac

thu1 = thulac.thulac()
# 只分词，过滤无意义词语，繁体转化为简体, 去掉原文的空格后再分词
thu2 = thulac.thulac(seg_only=True, filt=True, T2S=True, rm_space=True)

# thulac的模型载入时间比较久，并且每次运行会重复载入。为了避免这个问题所以
# 将模型载入和分词分开为两个单元格。

Model loaded succeed
Model loaded succeed


首先观察默认模式下的分词效果：
+ 对于简体中文，总体来说分词效果不错，但是却分错了一个很常见的词语“['公交', 'a'], ['车致', 'n']”。
+ 对于粤语，分词效果则依然很差。虽然相对于其他分词工具来说不偏向于分为单字，但是第2个粤语句子却完全没有分词；thulac可以正确划分一些词语，
比如“林鄭月娥”， “跌眼鏡”， “反口覆舌”。

其次，在自定义模式下：
+ 对于简体中文，自定义模式中并没有设置影响分词模型的参数，但是第2个句子却将原来分对的词语分错了（['工信', ''], ['处女', '']），可以说明
thulac中的filter参数可能并不是在默认模式分词的基础上过滤无意义词语，而是可能在分词之前过滤。详细过程需要进一步分析源代码。
+ 对于粤语，在自定义模式中设置了将繁体转化为简体。但是我们可以看到这个功能并不是根据语义转化，而应该是根据自定义的转化词典，将繁体简单的替换为简体，没有定义的粤语就不会转化，例如最后一句粤语。并且转化应该是在分词之前进行，因为默认模式下的“林鄭冇”在自定义模式下被划分为了“林鄭”和“没”，“嘅會係葉劉”也是同样。

总体来说，模型的分词效果受到一些自定义参数的影响。可能这些参数从理解上来说并不应该影响分词，但是最终的结果表明这些参数会较大的影响分词结果。而在默认模式下，thulac的分词效果总体好于SnowNLP，但是稍差于jieba。

In [8]:
for sent in sentences:
    print("默认模式：", thu1.cut(sent))
    print("自定义模式：", thu2.cut(sent))
    print("------")
    
for sent in cantonese_sents:
    print("默认模式：", thu1.cut(sent))
    print("自定义模式：", thu2.cut(sent))
    print("------")

默认模式： [['我', 'r'], ['来', 'v'], ['到', 'v'], ['北京', 'ns'], ['清华大学', 'ni']]
自定义模式： [['我', ''], ['来', ''], ['到', ''], ['北京', ''], ['清华大学', '']]
------
默认模式： [['工信处', 'n'], ['女', 'a'], ['干事', 'n'], ['每月', 'r'], ['经过', 'p'], ['下属', 'v'], ['科室', 'n'], ['都', 'd'], ['要', 'v'], ['亲口', 'd'], ['交代', 'v'], ['24', 'm'], ['口', 'q'], ['交换机', 'n'], ['等', 'u'], ['技术性', 'n'], ['器件', 'n'], ['的', 'u'], ['安装', 'v'], ['工作', 'v']]
自定义模式： [['工信', ''], ['处女', ''], ['干事', ''], ['下属', ''], ['科室', ''], ['都', ''], ['要', ''], ['亲口', ''], ['交代', ''], ['口', ''], ['交换机', ''], ['等', ''], ['技术性', ''], ['器件', ''], ['的', ''], ['安装', ''], ['工作', '']]
------
默认模式： [['探访', 'v'], ['“', 'w'], ['金特会', 'n'], ['”', 'w'], ['举办', 'v'], ['当天', 't'], ['越南', 'ns'], ['河内', 'ns'], ['街头', 's'], ['景象', 'n']]
自定义模式： [['探访', ''], ['“', ''], ['金特会', ''], ['”', ''], ['越南', ''], ['河内', ''], ['街头', ''], ['景象', '']]
------
默认模式： [['西青区', 'ns'], ['一', 'm'], ['“', 'w'], ['路', 'n'], ['怒女', 'n'], ['”', 'w'], ['早', 'a'], ['高峰', 'n'], ['别', 'd'], ['停',

thulac支持用户自定义词典，自定义词典使用文件的方式载入。从如下测试结果中可以观察到：
+ 设置只分词可能还会影响原来的分词结果，比如原来分对的“工信处”，“女”现在误分为“工信”,“处女”。
+ 分词的结果并不一定按照用户定义的方式划分，比如“洪荒之力”依然被分割，“林鄭月娥”没有被划分，“林“,”鄭”依然被分开。等等。

总体来说，thulac的分词结果很不稳定，容易受到设置参数和用户词典的影响，而分词结果却又不一定按照词典划分。

In [13]:
# 用户自定义词典
# 公交车
# 路怒女
# 洪荒之力
# 林鄭
# 馬騮戲
# 葉劉
# 係咪
# 工信处

thu3 = thulac.thulac(user_dict="user_dict.txt", seg_only=True)
for sent in sentences:
    print("用户词典模式：", [w[0] for w in thu3.cut(sent)])
    print("------")
    
for sent in cantonese_sents:
    print("用户词典模式：", [w[0] for w in thu3.cut(sent)])
    print("------")

Model loaded succeed
用户词典模式： ['我', '来', '到', '北京', '清华大学']
------
用户词典模式： ['工信', '处女', '干事', '每月', '经过', '下属', '科室', '都', '要', '亲口', '交代', '24', '口', '交换机', '等', '技术性', '器件', '的', '安装', '工作']
------
用户词典模式： ['探访', '“', '金特会', '”', '举办', '当天', '越南', '河内', '街头', '景象']
------
用户词典模式： ['西青区', '一', '“', '路怒女', '”', '早', '高峰', '别', '停', '公交车', '致', '多', '名', '乘客', '受伤', '！']
------
用户词典模式： ['西環', '出', '盡洪荒', '之', '力', '，', '林鄭月娥', '最', '終', '提名票', '是', '579', '，', '難免', '令', '外界', '跌眼鏡', '。']
------
用户词典模式： ['唔知', '係咪林', '鄭哋', '公關', '聽', '到']
------
用户词典模式： ['垃圾', '小圈子', '，', '當睇', '馬騮戲', ' ', '咁班', '犯', '民狗', '又', '會玩', '馬騮', '戲嘅', '？']
------
用户词典模式： ['大', '6', '戲', '一', '場', '。']
------
用户词典模式： ['如果', '林鄭冇', '反口', '覆舌', ',', ' ', '攞', '572', '票', '嘅會', '係葉', '劉']
------


## pyltp
pyltp==0.2.1
[pyltp](https://github.com/HIT-SCIR/pyltp)是[语言技术平台（Language Technology Platform, LTP）](https://github.com/HIT-SCIR/ltp)的python封装。语言技术平台由哈工大社会计算与信息检索研究中心研发，目前已经更新到3.4.0版本，并且推出了Docker镜像。pyltp支持个性化分词模型，在原来模型的基础上根据用户提供的少量标注数据完成增量训练。

In [3]:
import os
LTP_DATA_DIR = '/Users/yangqj/Documents/Workspace/Tools/pyltp_model/ltp_data_v3.4.0'
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')

from pyltp import Segmentor
segmentor = Segmentor()
segmentor.load(cws_model_path)

根据下面的测试结果可以观察到：
+ 对于简体中文，pyltp的分词效果总体还是不错的，大多数的分分词结果都比较正确。
+ 对于粤语来说，pyltp的分词效果是所有工具中最好的，比如争取划分了“西環”，“林鄭月娥”，
“跌眼鏡”， “馬騮戲”， “嘅 會 係 葉劉”。

In [5]:
for sent in sentences:
    print(" ".join(segmentor.segment(sent)))

for sent in cantonese_sents:
    print(" ".join(segmentor.segment(sent)))


我 来到 北京 清华大学
工信 处女 干事 每月 经过 下属 科室 都 要 亲口 交代 24 口 交换机 等 技术性 器件 的 安装 工作
探访 “ 金特会 ” 举办 当天 越南 河内 街头 景象
西青区 一 “ 路 怒女 ” 早 高峰 别 停 公交车 致 多 名 乘客 受伤 ！
西環 出 盡洪荒 之 力 ， 林鄭月娥 最 終 提名 票 是 579 ， 難免 令 外界 跌眼鏡 。
唔 知 係咪林 鄭哋 公關 聽 到
垃圾 小圈子 ， 當睇 馬 騮戲 咁班 犯 民狗 又 會 玩 馬騮戲 嘅 ？
大 6 戲 一 場 。
如果 林鄭冇 反口覆舌 , 攞 572 票 嘅 會 係 葉劉


从下面的测试结果来看，当载入用户自定义词典之后pyltp的分词效果进一步提升了：
+ 对于简体中文，“工信处 女”正确完成分词
+ 对于繁体，当同时加入"林鄭月娥"和“林鄭”，第1和第2句粤语都可以正确划分。且经过测试分词结果不受两个词语
前后顺序的影响
+ 加入用户自定义词典之后，原来的分词并没有收到影响，分词效果非常稳定。

In [11]:
# 用户自定义词典
# 公交车
# 路怒女
# 洪荒之力
# 林鄭月娥
# 林鄭
# 馬騮戲
# 葉劉
# 係咪
# 工信处


segmentor2 = Segmentor()
segmentor2.load_with_lexicon(cws_model_path, 'user_dict.txt')
for sent in sentences:
    print(" ".join(segmentor2.segment(sent)))

for sent in cantonese_sents:
    print(" ".join(segmentor2.segment(sent)))

我 来到 北京 清华大学
工信处 女 干事 每月 经过 下属 科室 都 要 亲口 交代 24 口 交换机 等 技术性 器件 的 安装 工作
探访 “ 金特会 ” 举办 当天 越南 河内 街头 景象
西青区 一 “ 路怒女 ” 早 高峰 别 停 公交车 致 多 名 乘客 受伤 ！
西環 出 盡 洪荒之力 ， 林鄭月娥 最 終 提名 票 是 579 ， 難免 令 外界 跌眼鏡 。
唔 知 係咪 林鄭 哋公 關聽 到
垃圾 小圈子 ， 當睇 馬騮戲 咁班 犯 民狗 又 會 玩 馬騮戲 嘅 ？
大 6 戲 一 場 。
如果 林鄭 冇 反口覆舌 , 攞 572 票 嘅 會 係 葉劉


In [12]:
segmentor.release()

## 总结
根据上面的测试结果，我们可以总结以下几点：
+ 总体来说，pyltp的效果是最好的。其原生分词效果对简体和繁体中文都不错，并且支持自定义词典，自定义训练等。
+ 在分词准确率上，大致可以排序如下pyltp > jieba > thulac > pynlpir > snownlp


| 工具    | 原理                                                         | 自定义词典 | 命令行/文件分词 | 训练模型                                                     | 分词结果 |
| ------- | ------------------------------------------------------------ | ---------- | --------------- | ------------------------------------------------------------ | -------- |
| 结巴    | 基于前缀词典生成有向无环图，再使用动态规划查找概率最大的划分。对于未登录词使用HMM模型。 | 支持       | 支持            | 基于前缀词典的分词只需要增加自定义的词频词典；HMM模型不支持训练。 | 2        |
| SnowNLP | [基于字的生成模型](http://aclweb.org/anthology//Y/Y09/Y09-2047.pdf) |            |                 | 支持                                                         | 5        |
| pynlpir | N最短路径方法得到粗分结果，对于未登录词使用HMM识别，在基于词性标注结果继续优化。 | 支持       | 支持            | 不支持                                                       | 4        |
| thulac  | [结构化感知器](http://www.cs.columbia.edu/~mcollins/papers/tagperc.pdf) | 支持       | 支持            | 支持                                                         | 3        |
| pyltp   | 结构化感知器                                                 | 支持       | 支持            | 在线训练                                                         | 1        |

[^结构化感知器]: Discriminative training methods for hidden markov models: Theory and experiments with perceptron algorithms



## PaperWeekly测评

PaperWeekly对上述除SnowNLP之外的4中工具进行了[评测](http://rsarxiv.github.io/2016/11/29/%E4%B8%AD%E6%96%87%E5%88%86%E8%AF%8D%E5%B7%A5%E5%85%B7%E6%B5%8B%E8%AF%84/)，结果如下：

MSR测试结果
![](http://rsarxiv.github.io/2016/11/29/%E4%B8%AD%E6%96%87%E5%88%86%E8%AF%8D%E5%B7%A5%E5%85%B7%E6%B5%8B%E8%AF%84/media/1.png)

PKU测试结果
![](http://rsarxiv.github.io/2016/11/29/%E4%B8%AD%E6%96%87%E5%88%86%E8%AF%8D%E5%B7%A5%E5%85%B7%E6%B5%8B%E8%AF%84/media/2.png)

人民日报测试结果
![](http://rsarxiv.github.io/2016/11/29/%E4%B8%AD%E6%96%87%E5%88%86%E8%AF%8D%E5%B7%A5%E5%85%B7%E6%B5%8B%E8%AF%84/media/3.png)

1、一个好的分词工具不应该只能在一个数据集上得到不错的指标，而应该在各个数据集都有很不错的表现。从这一点来看，thulac和ltp都表现非常不错。

2、因为分词是个基础部件，分词速度对于一个分词工具来说也至关重要。从这一点来看，thulac和jieba表现的不错。

3、大家都知道，基本的分词依赖模型，但真正想用分词工具来解决应用层面上的问题，都需要借助于词库，本文测试的4个工具均支持用户自定义词库。

4、特别需要强调的一点是，哈工大的ltp支持分词模型的在线训练，即在系统自带模型的基础上可以不断地增加训练数据，来得到更加丰富、更加个性化的分词模型。
